In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import torch

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

## 에이다 부스트
에이다부스트는 위원회에 넣을 개별 모형 km을 선별하는 방법으로는 학습 데이터 집합의 i번째 데이터에 가중치 wi를 주고 분류 모형이 틀리게 예측한 데이터의 가중치를 합한 값을 손실함수 L로 사용한다. 이 손실함수를 최소화하는 모형이 km으로 선택된다.

In [ ]:
from sklearn.datasets import make_gaussian_quantiles
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier


x1,y1=make_gaussian_quantiles(
    cov=2, n_samples=100,n_features=2,
    n_classes=2,random_state=1
    )

x2,y2=make_gaussian_quantiles(
    mean=(3,3),cov=1.5,
    n_samples=200,n_features=2,
    n_classes=2,random_state=1
    )

X=np.concatenate((x1,x2))
Y=np.concatenate((y1, - y2 + 1))
class MyAdaBoostClassifier(AdaBoostClassifier):
    
    def __init__(self,
                 base_estimator=None,
                 n_estimators=50,
                 learning_rate=1.,
                 algorithm='SAMME.R',
                 random_state=None):

        super(MyAdaBoostClassifier, self).__init__(
            base_estimator=base_estimator,
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            random_state=random_state)
        self.sample_weight = [None] * n_estimators
        
    def _boost(self, iboost, X, y, sample_weight, random_state):
        sample_weight, estimator_weight, estimator_error = \
        super(MyAdaBoostClassifier, self)._boost(iboost, X, y, sample_weight, random_state)
        self.sample_weight[iboost] = sample_weight.copy()
        return sample_weight, estimator_weight, estimator_error
    
model_ada = MyAdaBoostClassifier(DecisionTreeClassifier(max_depth=1, random_state=0), n_estimators=20)
model_ada.fit(X,Y)

In [ ]:

def plot_result(model, title="분류결과", legend=False, s=50):
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, 0.02), np.arange(x2_min, x2_max, 0.02))
    if isinstance(model, list):
        Y = model[0].predict(np.c_[xx1.ravel(), xx2.ravel()]).reshape(xx1.shape)
        for i in range(len(model) - 1):
            Y += model[i + 1].predict(np.c_[xx1.ravel(), xx2.ravel()]).reshape(xx1.shape)
    else:
        Y = model.predict(np.c_[xx1.ravel(), xx2.ravel()]).reshape(xx1.shape)
    cs = plt.contourf(xx1, xx2, Y, cmap=plt.cm.Paired, alpha=0.5)
    for i, n, c in zip(range(2), "01", "br"):
        idx = np.where(y == i)
        plt.scatter(X[idx, 0], X[idx, 1], c=c, s=s, alpha=0.5, label="Class %s" % n)
    plt.xlim(x1_min, x1_max)
    plt.ylim(x2_min, x2_max)
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.title(title)
    plt.colorbar(cs)
    if legend:
        plt.legend()
    plt.grid(False)

plot_result(model_ada, "AdaBoost(m=20) Classification Result")

## 에이다부스트 모형의 정규화
에이다부스트 모형이 과최적화가 되는 경우에는 학습 속도(learning rate) 조정하여 정규화를 할 수 있다. 이는 필요한 멤버의 수를 강제로 증가시켜서 과최적화를 막는 역할을 한다.

에이다부스트 모형이 과최적화가 되는 경우에는 학습 속도(learning rate) 조정하여 정규화를 할 수 있다. 이는 필요한 멤버의 수를 강제로 증가시켜서 과최적화를 막는 역할을 한다.

Cm=Cm−1+μαmkm

AdaBoostClassifier 클래스에서는 learning_rate 인수를 1보다 적게 주면 새로운 멤버의 가중치를 강제로 낮춘다.



In [ ]:
%%time 

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score

mean_test_accuracy = []
for n in np.arange(1, 1001, 100):
    model1 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=n)
    mean_test_accuracy.append(cross_val_score(model1, X, Y, cv=5).mean())

In [ ]:
plt.plot(np.arange(1, 1000, 100), mean_test_accuracy)
plt.show()

## 그레디언트 부스트
그레디언트 부스트 모형에서는 손실 범함수(loss functional) L(y,Cm−1)을 최소화하는 개별 분류함수 km를 찾는다. 이론적으로 가장 최적의 함수는 범함수의 미분이다.

따라서 그레디언트 부스트 모형은 분류/회귀 문제에 상관없이 개별 멤버 모형으로 회귀분석 모형을 사용한다. 가장 많이 사용되는 회귀분석 모형은 의사결정 회귀나무(decision tree regression model) 모형이다.

그레디언트 부스트 모형에서는 다음과 같은 과정을 반복하여 멤버와 그 가중치를 계산한다.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
model_grad=GradientBoostingClassifier(
    n_estimators=100, max_depth=2, random_state=0
)

%time
model_grad.fit(X,Y)

In [ ]:
plot_result(model_grad)

In [ ]:
plot_result(model_grad.estimators_[0][0])

In [ ]:
## XGBoost
import xgboost
model_xgb = xgboost.XGBClassifier(n_estimators=100, max_depth=1, random_state=0)

In [ ]:
model_xgb.fit(X,Y)
plot_result(model_xgb)

In [ ]:
## lightgbm
import lightgbm

model_lgbm = lightgbm.LGBMClassifier(n_estimators=100, max_depth=1, random_state=0)
%time
model_lgbm.fit(X, y)
plot_result(model_lgbm)